# 크롤러 환경설정

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json
import os, sys
import tweepy

sys.path.append(os.pardir)
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
key_path = 'C:/Users/dkdud/Untitled Folder/api_key.json'
with open(key_path,'r') as f:
    key = json.load(f)

# 개인정보 인증을 요청하는 Handler입니다.
auth = tweepy.OAuthHandler(key['CONSUMER_KEY'], key['CONSUMER_SECRET'])

# 인증 요청을 수행합니다.
auth.set_access_token(key['ACCESS_TOKEN_KEY'], key['ACCESS_TOKEN_SECRET'])

# twitter API를 사용하기 위한 준비입니다.
api = tweepy.API(auth,wait_on_rate_limit=True)

In [5]:
#인스타 게시물을 불러오기 위한 사전작업 단계
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='data.campuss'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'data.2021'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)

def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

    
def get_content(driver):
    
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    tag_list =[]
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content, tag_data]
    return data

In [6]:
def insta_crawling(driver,keyword, target):
    
    result_list =[]
    url = insta_searching(keyword)
    driver.implicitly_wait(2)
    driver.get(url)
    time.sleep(2)

    click_first(driver)

    for i in range(target):
        try:
            result_list.append(get_content(driver))
            next_page(driver)
        except:
            time.sleep(2)
            next_page(driver)
    df=pd.DataFrame(result_list)
    df.columns=['본문','해시태그']
    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : str(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : clean_text(x))
    df['본문']= df['본문'] + df['해시태그']
    df.drop(['해시태그'],axis=1,inplace=True)
    df = df.drop_duplicates(subset=['본문'])
    return df

In [7]:
#트위터 게시물 가져오는 단계
def twitter_crawling(keyword):
    columns = ['본문']
    df = pd.DataFrame(columns=columns)

    max_tweets = 1000
    searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]

    for tweet in searched_tweets:
        tweet_json = tweet._json
        tweet_text = tweet_json['text']
        row = [tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df = df.drop_duplicates(subset=['본문'])
    
    return df


In [8]:
keyword = '환경오염'
target = 200

In [9]:
insta_df = insta_crawling(driver,keyword, target)

In [10]:
twitter_df = twitter_crawling(keyword)

In [95]:
sns_df = pd.concat([twitter_df, insta_df],join='outer',ignore_index=True)

In [100]:
sns_df.to_csv('환경.csv',encoding='UTF-8-SIG')

In [101]:
text= []
for x in range(0,256):
    try:
        tp = sns_df['본문'][x]
        tp = tp.split('.')
    except:
        print("0")
    text.append(tp)

In [102]:
import itertools
text=list(itertools.chain(*text))

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm_notebook
import re

In [111]:
data= pd.read_csv('환경오염 크롤링.csv',encoding='UTF-8-SIG', index_col=0)
data = data.dropna()

okt = Okt()
noun_list =[]
for content in data['본문']:
    nouns = okt.nouns(content)
    noun_list. append(nouns)

data['명사'] = noun_list
data

,본문,명사
0,모든 남자아이돌 옷장과 협찬에서 비니와 벙거지를 없애주세요 환경오염입니다 지구야미안해,"[모든, 남자, 아이돌, 옷장, 협찬, 비니, 벙거지, 환경오염, 지구]"
1,쓰레기를 버리지말자는 주장에 환경오염과 환경미화원의 노고를 생각하면 옳은비판이었다고...,"[쓰레기, 주장, 환경오염, 환경미화원, 노고, 생각, 비판, 논쟁, 종료, 사람,..."
2,환경오염때문에 온거면 환경주제도꺼낼수잇어요,"[환경오염, 때문, 환경, 주제]"
3,그래서 쓰레기를 무단투기하지 말라는 주장이 타집단에서 나오면 그게 옳다고 동의하는게...,"[쓰레기, 무단, 투기, 말, 주장, 집단, 그게, 동의, 쓰레기, 무단, 투기, ..."
4,공룡행성에서 왜왓어요 환경오염때문인가요,"[공룡, 행성, 왜왓어, 환경오염, 때문]"
...,...,...
368,도가 넘는 고열에도 타지 않고자연분해 되는데 무려 년 이 걸리는쌓여만가던 젤 아이스...,"[도, 고열, 타지, 자연, 분해, 무려, 년, 이, 젤, 아이스팩, 이제, 고민,..."
369,바다 바라 바래 보다 프로젝트리서치트립 기록 월 저녁무렵 바다 바라 바래 보다 프로...,"[바다, 바, 프로젝트, 리, 치트, 립, 기록, 월, 저녁, 무렵, 바다, 바, ..."
370,우리는 끝도 없는 쓰레기라는 팬데믹에 직면했습니다 갈수록 늘어나는 쓰레기 때문에 매...,"[우리, 끝, 쓰레기, 팬데믹, 직면, 갈수록, 쓰레기, 때문, 매립, 포화상태, ..."
371,한살림자주모임 한살림경기동부식생활양성과정 함께 했던 경기동부분들과 한달에 한번 지속...,"[한살림, 자주, 모임, 한살림, 경기, 동부, 식생활, 양성, 과정, 동부, 달,..."


In [89]:
data.to_csv('환경오염크롤링중.csv',encoding='UTF-8-SIG')

In [80]:
df = pd.DataFrame(sns_df,columns=['본문'])

In [81]:
drop_index = df[df['본문']==""].index
df.drop(drop_index,inplace=True)

In [72]:
from eunjeon import Mecab
tokenizer = Mecab()
pos_temp = {}
pos_list = []

for i in df['본문']:
    temp_str = ''
    try:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]+=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    except:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    pos_list.append(temp_str)

KeyboardInterrupt: 